# Mask tidier from server

This notebook performs same mask post-processing but pulls masks already segmented and saved on server and then tidies them and saves them locally. 

In [39]:
import glob, os
import numpy as np
from skimage.io import imread, imsave
from skimage.morphology import remove_small_objects, remove_small_holes
from skimage.measure import label, regionprops
from tqdm.auto import tqdm
from natsort import natsorted

Just doing ND0025 for now

In [19]:
files = glob.glob('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/preprocessed/ND0025/Pos*/Pos*_images/*channel099*')

In [20]:
len(files)

11839

In [52]:
for fn in tqdm(natsorted(files)):
    mask = imread(fn)
    mask_orig = mask
    gfp_mask = mask == 1
    rfp_mask = mask == 2
    
    rfp_mask = remove_small_objects(rfp_mask, min_size=150).astype(np.uint8)*2
    gfp_mask = label(remove_small_objects(remove_small_holes(gfp_mask), min_size=150))

    for i in range(1, np.amax(gfp_mask)+1):
        #DOEESNT- already eroded ## needs erosion step to stop merging of labels
        segment = gfp_mask == i
        seg_props = regionprops(label(segment))[0]
        if 3000 <= seg_props.area or seg_props.eccentricity > 0.95 or seg_props.area < (0.9*seg_props.filled_area):
            #segment = binary_erosion(segment)
            gfp_mask[segment] = 3
        else:
            #segment = binary_erosion(segment)
            gfp_mask[segment] = 1
    
    mask = gfp_mask.astype(np.uint8) + rfp_mask.astype(np.uint8)
    
    imsave(fn, mask.astype(np.uint8), check_contrast=False)
    

  0%|          | 0/11839 [00:00<?, ?it/s]

# Once the above has finished then copy whole folder across to local drive

If it fails then can do if 3 in mask to find already processed images

## Realised I need to copy everything across because I haven't filtered out local blanks

In [21]:
### first copy transformation matrices across
trans_files = glob.glob('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/preprocessed/ND0025/Pos*/*.npy')

In [24]:
server_path = '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/preprocessed/'
local_path = '/home/nathan/data/kraken/ras/'

In [25]:
fn.replace(server_path, local_path)

'/home/nathan/data/kraken/ras/ND0025/Pos7/transform_tensor.npy'

In [22]:
for fn in trans_files:
    fn = fn.replace(server_path, local_path)

['/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/preprocessed/ND0025/Pos7/transform_tensor.npy',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/preprocessed/ND0025/Pos5/transform_tensor.npy',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/preprocessed/ND0025/Pos12/transform_tensor.npy',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/preprocessed/ND0025/Pos4/transform_tensor.npy',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/preprocessed/ND0025/Pos13/transform_tensor.npy',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/preprocessed/ND0025/Pos11/transform_tensor.npy',
 '/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data